# V20 diffraction experiment - February 2018
Objective: plotting and comparing experimental data measured on DENEX detector (.root files) and He3 tubes (.asc files) and results of McStas simulations.

## Open output of McStas simulation
This method uses the datafiles generated by a `McStas` simulation.

In [ ]:
import os 
from mcstasscript.interface import instr, plotter, functions, reader

import numpy as np
import matplotlib.pyplot as plt

import ipywidgets

%matplotlib inline

In [ ]:
#  path to load McStas simulation data 
path_to_model = '/Users/celinedurniak/V20DiffractionData'

In [ ]:
# Folder containing output of McStas simulation
result_folder = 'V20_config6'

In [ ]:
# access data stored after McStas simulations
data_to_plot = functions.load_data(os.path.join(path_to_model, result_folder))

In [ ]:
# Create a dictionary containing filenames, shape of output data and x, y labels
dict_mcstas_files={}

for item in data_to_plot:
    file_key = item.metadata.info['filename'].rstrip()
    xlabel = item.metadata.info['xlabel'].rstrip()
    ylabel = item.metadata.info['ylabel'].rstrip()
    type_array = item.metadata.info['type']
    start = type_array.find('(') + 1
    end = type_array.find(')', start)
    
    if ',' in type_array[start:end]:
        nx_value, ny_value = type_array[start:end].split(',')
        
        dict_mcstas_files[file_key] = ((int(nx_value),
                                        int(ny_value)), 
                                       xlabel, 
                                       ylabel)
    else:
        dict_mcstas_files[file_key] = (int(type_array[start:end]), 
                                       xlabel, 
                                       ylabel)
        
dict_mcstas_files

In [ ]:
# #  plot 1D data generated by McStas simulation and plot using matplotlib
selected_filename = 'monitor_Hetube5.dat'

print(dict_mcstas_files[selected_filename])

fig, ax = plt.subplots()

x_mcstas, y_mcstas = np.genfromtxt(os.path.join(path_to_model,result_folder, selected_filename), 
                                   usecols=(0,1),unpack=True)
ax.plot(x_mcstas, y_mcstas, label=selected_filename)
ax.set_xlabel(dict_mcstas_files[selected_filename][1])
ax.set_ylabel(dict_mcstas_files[selected_filename][2])
ax.legend()
ax.grid()

In [ ]:
# plot 2D - the number of line to read is hard-coded but could be accessed from the header of the file: '# type: array_2d(150, 150)'
selected_filename = 'monitor_tx_DENEX.dat'
print(dict_mcstas_files[selected_filename])
# check that we are dealing with 2D data and set the limit to read the first matrix only
data2d = np.genfromtxt(os.path.join(path_to_model, result_folder, selected_filename), 
                       max_rows=dict_mcstas_files[selected_filename][0][0])

fig, ax = plt.subplots(figsize=(8, 8))
contf = ax.imshow(np.flip(data2d, 0), aspect='auto')
ax.set_xlabel(dict_mcstas_files[selected_filename][1])
ax.set_ylabel(dict_mcstas_files[selected_filename][2])
ax.set_title(selected_filename)
cbar = fig.colorbar(contf)

Since the sample used for McStas simulations is Nacalf, the possible ROOT files and He3 tubes to use are:
- Spectrum03
- Spectrum11
- Spectrum12

## He3 data

These are the files with an '.asc' extension.

*Naming of these files:*
-  The `bn4` format is a raw data file (event mode) that can be rebinned. Christian Jacobsen (HZG) wrote us a small python script to generate asci files from it, which also generates the filename. So you are completely right: the number indicates the binning size used to histogram the data. I attach the python script as well.
- `chX` corresponds to the channel # of the 4 tubes.  
    a. There are four tubes for diffraction, *i.e.* four channels. TsDau has a total of eight channels. Two of them were used for beam monitors, two were empty.  
    b.  In our case (Feb 2018), `Ch3` was a beam monitor and `Ch5`, `Ch6`, `Ch7`, `Ch8` were the four diffraction tubes.
- `binX` indicates the number of bins used to histogram the data. 

Only data with 2500 bins were considered for Spectrum3 in order to be consistent for all spectra.

In [ ]:
path_to_he3_files = '/Users/celinedurniak/V20DiffractionData/TsDau'

In [ ]:
# Spectrum 3
#he3sp3ch3_x, he3sp3ch3_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum03.bn4ch3_bin2500.asc'), unpack=True)
he3sp3ch5_x, he3sp3ch5_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum03.bn4ch5_bin2500.asc'), unpack=True)
#he3sp3ch6_x, he3sp3ch6_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum03.bn4ch6_bin2500.asc'), unpack=True)
#he3sp3ch7_x, he3sp3ch7_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum03.bn4ch7_bin2500.asc'), unpack=True)
#he3sp3ch8_x, he3sp3ch8_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum03.bn4ch8_bin2500.asc'), unpack=True)

In [ ]:
# # Spectrum 11
# he3sp11ch3_x, he3sp11ch3_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum11.bn4ch3_bin2500.asc'), unpack=True)
# he3sp11ch5_x, he3sp11ch5_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum11.bn4ch5_bin2500.asc'), unpack=True)
# he3sp11ch6_x, he3sp11ch6_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum11.bn4ch6_bin2500.asc'), unpack=True)
# he3sp11ch7_x, he3sp11ch7_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum11.bn4ch7_bin2500.asc'), unpack=True)
# he3sp11ch8_x, he3sp11ch8_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum11.bn4ch8_bin2500.asc'), unpack=True)

In [ ]:
 # Spectrum 12
# he3sp12ch3_x, he3sp12ch3_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum12.bn4ch3_bin2500.asc'), unpack=True)
# he3sp12ch5_x, he3sp12ch5_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum12.bn4ch5_bin2500.asc'), unpack=True)
# he3sp12ch6_x, he3sp12ch6_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum12.bn4ch6_bin2500.asc'), unpack=True)
# he3sp12ch7_x, he3sp12ch7_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum12.bn4ch7_bin2500.asc'), unpack=True)
# he3sp12ch8_x, he3sp12ch8_y = np.genfromtxt(os.path.join(path_to_he3_files, 'Spectrum12.bn4ch8_bin2500.asc'), unpack=True)

## ROOT files
Information about 2D data stored in ROOT files

| Spectrum   | $x_{min}$ | $x_{max}$ | $y_{min}$ | $y_{max}$ |
| ---------- | :-------- | :-------- | :-------- | :-------- |
| Spectrum03 | 0.0       | 1024      | -4608     | 4608      |
| Spectrum11 | 0.0       | 4096      | -4608     | 4608      |
| Spectrum12 | 0.0       | 4096      | -4608     | 4608      |

In [ ]:
# Create dictionary to generate plots
# Each entry corresponds to the spectrum number, the associated root file and the folder to get the data from
dict_root_files = {'Spectrum03': ('Spectrum03_DENEX006_1_18-02-05_0000.root', 'Meas_3'), 
                   'Spectrum11': ('Spectrum11_DENEX006_1_18-02-09_0001.root', 'Meas_1'),
                   'Spectrum12': ('Spectrum12_DENEX006_1_18-02-10_0000.root', 'Meas_1')}

dict_selected_dataset = {
'Spectrum03': ('H_TOF_dsp_after_run_3', 'H_TOF,X1-X2_User_2D2_dsp_after_run_3'),
'Spectrum11': ('H_TOF_User_1D1_dsp_after_run_1', 'H_TOF,X1-X2_User_2D4_dsp_after_run_1'),
'Spectrum12': ('H_TOF_User_1D1_dsp_after_run_1', 'H_TOF,X1-X2_User_2D4_dsp_after_run_1')
}

In [ ]:
import uproot

path_to_root_file = '/Users/celinedurniak/V20DiffractionData/DENEX'

In [ ]:
# open one ROOT file and extract only one 1D and one 2D dataset specified in dict_selected_dataset
# The 1D and 2D datasets are stored in a dictionary
# Note the vertical axis of 2D datasets is inverted

dict_output = {}

key_spectrum ='Spectrum03'
    
ROOTfile = dict_root_files[key_spectrum][0]
dir_with_data = dict_root_files[key_spectrum][1]

file_to_open = os.path.join(path_to_root_file, ROOTfile)

with uproot.open(file_to_open)[dir_with_data] as myFile:
    for key in myFile.keys():
        # 1D line plot
        if 'TH1I' in str(myFile[key]) and dict_selected_dataset[key_spectrum][0] in str(key):     
            key_name = myFile[key].name.decode('utf-8')   
            dict_output[key_spectrum+'_1D'] = myFile[key].values

        # 2D contourplot
        elif 'TH2' in str(myFile[key]) and dict_selected_dataset[key_spectrum][1] in str(key):
            key_name = myFile[key].name.decode('utf-8') 
            dict_output[key_spectrum+'_2D'] = arr_object = np.flip(myFile[key].values, 1)                 

# Comparison between data from He3, DENEX and McStas simulation

We use the 1D datasets loaded in the previous cells.

## Physical units 

|        | Time          | Length | Intensity     |
| ------ | :------------ | :----- | :------------ |
| McStas | seconds       | meter  | neutron/s/bin |
| He3    | milliseconds  | ?      | neutron/bin?  |
| ROOT   | ?             | ?      | ?             |

### Notes: Conversion of McStas 1D data

$$\begin{eqnarray}
x \rightarrow a_{x\_mcstas} x + b_{x\_mcstas} \\
y \rightarrow a_{y\_mcstas} y + b_{y\_mcstas} \\
\end{eqnarray}$$

A similar convention is also used to rescale data from ROOT files and He3tubes.

In [ ]:
# define colors to be used for the plots

colors_curves = ['#1f77b4', '#ff7f0e', '#2ca02c']
# Mcstas data - store initial values
xini_mcstas = x_mcstas
yini_mcstas =  y_mcstas 

# ROOT data H_TOF_dsp_after_run_3
y_root = dict_output['Spectrum03_1D']
x_root = np.arange(len(y_root))
# store initial values
xini_root = x_root
yini_root = y_root 

# He3 data - store initial values
xini_he3 = he3sp3ch5_x
yini_he3 = he3sp3ch5_y

In [ ]:
# Define function to plot when scaling is updated
def fct_scaling(formula_xm, formula_ym, 
                formula_xr, formula_yr, 
                formula_xh, formula_yh, 
                show_mcstas, show_root, show_he3):
    
    fig, ax = plt.subplots(figsize=(10,10))

    x_mcstas = [eval(formula_xm) for x in xini_mcstas]
    y_mcstas = [eval(formula_ym) for y in yini_mcstas]
    
    x_root = [eval(formula_xr) for x in xini_root]
    y_root = [eval(formula_yr) for y in yini_root]
    
    x_he3 = [eval(formula_xh) for x in xini_he3]
    y_he3 = [eval(formula_yh) for y in yini_he3]
    
    line_mcstas, = ax.plot(x_mcstas, y_mcstas, label='mcstas')
    line_mcstas.set_visible(show_mcstas)
    
    line_root, = ax.plot(x_root, y_root, label='root')
    line_root.set_visible(show_root)
    
    line_he, = ax.plot(x_he3, y_he3, label='he3')
    line_he.set_visible(show_he3)
    
    ax.legend()
    ax.grid()
    plt.plot()

# set textboxes for the scaling formulas    
description_textbox = 'Enter a scaling formula'
style_textbox = {'description_width': 'initial'}
formula_xm = ipywidgets.Text(value='x',
                             placeholder=description_textbox,
                             description=r'\(\alpha_{x,m}*x+\beta_{x,m}\)', 
                             style=style_textbox,
                             description_tooltip="Scaling formula for x axis of McStas data",
                             continuous_update=False) 

formula_ym = ipywidgets.Text(value='y',
                             placeholder=description_textbox,
                             description=r'\(\alpha_{y,m}*y+\beta_{y,m}\)', 
                             style=style_textbox,
                             description_tooltip="Scaling formula for y axis of McStas data",
                             continuous_update=False) 

formula_xr = ipywidgets.Text(value='x',
                             placeholder=description_textbox,
                             description=r'\(\alpha_{x,r}*x+\beta_{x,r}\)', 
                             style=style_textbox,
                             description_tooltip="Scaling formula for x axis of ROOT data",
                             continuous_update=False) 

formula_yr = ipywidgets.Text(value='y',
                             placeholder=description_textbox,
                             description=r'\(\alpha_{y,r}*y+\beta_{y,r}\)', 
                             style=style_textbox,
                             description_tooltip="Scaling formula for y axis of ROOT data",
                             continuous_update=False) 

formula_xh = ipywidgets.Text(value='x',
                             placeholder=description_textbox,
                             description=r'\(\alpha_{x,h}*x+\beta_{x,h}\)', 
                             style=style_textbox,
                             description_tooltip="Scaling formula for x axis of He3 data",
                             continuous_update=False)

formula_yh = ipywidgets.Text(value='y',
                             placeholder=description_textbox,
                             description=r'\(\alpha_{y,h}*y+\beta_{y,h}\)', 
                             style=style_textbox,
                             description_tooltip="Scaling formula for y axis of He3 data",
                             continuous_update=False) 

# checkboxes to hide curves
hide_mcstas_curve = ipywidgets.Checkbox(value=True,
    description='Show mcStas curve',
    disabled=False,
    indent=False)

hide_root_curve = ipywidgets.Checkbox(value=True,
    description='Show ROOT curve',
    disabled=False,
    indent=False)

hide_he3_curve = ipywidgets.Checkbox(value=True,
    description='Show he3 curve',
    disabled=False,
    indent=False)

# group widgets horizontally
grid_mcstas = ipywidgets.HBox([formula_xm, formula_ym, hide_mcstas_curve], 
                              layout={'border': f'1.5px solid {colors_curves[0]}'}) 

grid_root = ipywidgets.HBox([formula_xr, formula_yr, hide_root_curve], 
                            layout={'border': f'2px solid {colors_curves[1]}'})

grid_he = ipywidgets.HBox([formula_xh, formula_yh, hide_he3_curve], 
                          layout={'border': f'2px solid {colors_curves[2]}'})

# reset button
reset_btn = ipywidgets.Button(description='Reset', 
                              button_style='info', 
                              tooltip='Click to reset to initial values')

def on_reset(event):
    """Reset the interactive plots to inital values."""
    formula_xm.value ='x'
    formula_xr.value ='x'
    formula_xh.value ='x'
    formula_ym.value ='y'
    formula_yr.value ='y'
    formula_yh.value ='y'
    
# reset textboxes to initial values when clicking on Reset button
reset_btn.on_click(on_reset) 

# Display plots and widgets
grid_text = ipywidgets.VBox([grid_mcstas, grid_root, grid_he, reset_btn]) 

interactive_plot = ipywidgets.interactive_output(fct_scaling, 
                                {'formula_xm': formula_xm, 'formula_ym': formula_ym, 
                                 'formula_xr': formula_xr, 'formula_yr': formula_yr, 
                                 'formula_xh': formula_xh, 'formula_yh': formula_yh,
                                 'show_mcstas': hide_mcstas_curve,
                                 'show_root': hide_root_curve, 
                                 'show_he3': hide_he3_curve}
                                ) 

display(grid_text, interactive_plot)